In [1]:
import numpy as np
import json

In [17]:
# save H4 as numpy array

f = open("data/h4/H4_Hinde_QZ.txt")
data = json.load(f)
H4 = np.array(data['data'])
np.savetxt("data/h4/h4_QZ_data.txt", H4)

f = open("data/h4/H4_Hinde_TZ.txt")
data = json.load(f)
H4 = np.array(data['data'])
np.savetxt("data/h4/h4_TZ_data.txt", H4)





TZ


In [24]:
# transform to coordinates:
np.loadtxt("data/h4/h4_QZ_data.txt")

array([[ 1.1       ,  4.25      ,  3.15      ,  5.35      ,  4.25      ,
         1.1       ],
       [ 1.1       ,  4.10736224,  3.33385294,  5.20346267,  4.42820237,
         1.1       ],
       [ 1.1       ,  3.74065502,  3.74065502,  4.83140766,  4.83140766,
         1.1       ],
       ...,
       [ 1.7       , 12.        , 12.11981848, 12.11981848, 12.        ,
         1.7       ],
       [ 1.7       , 12.01762167, 12.10234561, 12.10234561, 12.01762167,
         1.7       ],
       [ 1.7       , 12.06005804, 12.06005804, 12.06005804, 12.06005804,
         1.7       ]])